Here is my extended search function fromte text processing assignment

In [8]:
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

%load_ext jupyternotify

# test
#%notify
#import time
#time.sleep(5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
import re, glob, os
from collections import defaultdict
from pathlib import Path
import pandas as pd


In [10]:
# ver. 2 - uses a list of regex terms.

### NOTE ###
# - The code counts terms twice, if they are similar within the same pass. This means that drunk* and drun* count 1 and 2, because the loop counts drunk* again on the second iteration. THIS IS A PROBLEM WITH DRUNK* AND DRUNKARD!!!
# -  

def regex_search(folder_path , regList, verbose=1):
    """
    Search a folder path for a given regex.

    :folder_path: a string with the path to a dir with processed .txt files 
    :regList: a list of regular expressions as strings.
    """
    resultsDict = defaultdict(lambda: defaultdict(dict)) #consider using regular dict for the inner layer, since you know which terms you are searching.


    file_names = []

    # compile any search term(s) given for searching.
    #search_terms = re.compile('|'.join(regList)) # with word boundaries: re.compile(r'\b(?:%s)\b' % '|'.join(regList))
    search_terms_re = [re.compile(reg) for reg in regList]


    #loop through the folder with txt files.
    for filepath in glob.iglob(folder_path + "/*"):  #for future reference, if you just want to iterate through a number of files, use os.listdir() and slice it [:20] for instance.
        n_hits = 0
        filename = os.path.basename(filepath)
        #file_names.append(filename)
        
        #open the file
        with open(filepath, "r") as infile:
            content = infile.read()

        #find the document length (total tokens)
        resultsDict[filename]['text_length'] = len(re.findall(r'\w+', content))

        #search for the terms in the file, given the list of regexes.
        for term in search_terms_re:
            if term.findall(content):
                n_hits += 1
           

        # add the coutn of hits to the nested results dictionary 
            resultsDict[filename][term.pattern] = n_hits


    #export to df
    resultsDF = pd.DataFrame.from_dict(resultsDict, orient='index')
    # (re)name the index column
    #resultsDF = resultsDF.rename_axis('file_id')

    return resultsDF

#print(output)
#print(list(results.items())[:10])


#### for this code I used solutions from:
# https://stackoverflow.com/questions/6750240/how-to-do-re-compile-with-a-list-in-python
# https://howchoo.com/g/yjjknjdinmq/nested-defaultdict-python
# https://stackoverflow.com/questions/19851005/rename-pandas-dataframe-index 
# https://www.geeksforgeeks.org/python-program-to-count-words-in-a-sentence/


In [11]:
searchQuery = [
                "drunk*",
                "intoxicat*",
                "temperance",
                "beer*",
                "liquor",
                "alcohol*",
                "brandy",
                "booz*",
                "drunkard",
                "by-drink*"]

In [12]:

#function call

output = regex_search("C:/TEMPORARY/18" , searchQuery)
%notify

<IPython.core.display.Javascript object>

In [15]:
# inspect the dataframe
output

,text_length,drunk*,intoxicat*,temperance,beer*,liquor,alcohol*,brandy,booz*,drunkard,by-drink*
1800-01-15_f18000115-1-person44.txt,249,0,0,0,0,0,0,0,0,0,0
1800-01-15_s18000115-1-person834.txt,295,0,0,0,0,0,0,0,0,0,0
1800-01-15_t18000115-1-verdict5.txt,1660,1,2,2,3,4,4,4,4,4,4
1800-01-15_t18000115-10-verdict52.txt,3373,0,0,0,1,2,2,2,3,3,3
1800-01-15_t18000115-11-punish57.txt,415,0,0,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
1899-12-11_t18991211-86-punishment-44.txt,1453,0,0,0,1,1,1,1,1,1,1
1899-12-11_t18991211-87-punishment-45.txt,41,0,0,0,0,0,0,0,0,0,0
1899-12-11_t18991211-name-7.txt,309,0,0,0,1,1,1,1,1,1,1
1899-12-11_t18991211-name-98.txt,21,0,0,0,0,0,0,0,0,0,0


In [18]:
# write the output to a human friendly files

output.to_csv('results.tsv', sep="\t", index_label='file_id')
output.to_excel('results.xlsx', index_label='file_id')

In [3]:
# read the output of the last run, drop the index column.
df_in = pd.read_csv('results.tsv', sep='\t', index_col=0)

In [4]:
# add all values for the term columns
df_in['terms_sum'] = df_in.iloc[:, 1:].sum(axis=1)  # https://stackoverflow.com/questions/48923460/how-do-i-sum-a-column-range-in-python
df_in

,text_length,drunk*,intoxicat*,temperance,beer*,liquor,alcohol*,brandy,booz*,drunkard,by-drink*,terms_sum
file_id,,,,,,,,,,,,
1800-01-15_f18000115-1-person44.txt,249,0,0,0,0,0,0,0,0,0,0,0
1800-01-15_s18000115-1-person834.txt,295,0,0,0,0,0,0,0,0,0,0,0
1800-01-15_t18000115-1-verdict5.txt,1660,1,2,2,3,4,4,4,4,4,4,32
1800-01-15_t18000115-10-verdict52.txt,3373,0,0,0,1,2,2,2,3,3,3,16
1800-01-15_t18000115-11-punish57.txt,415,0,0,0,1,1,1,1,1,1,1,7
...,...,...,...,...,...,...,...,...,...,...,...,...
1899-12-11_t18991211-86-punishment-44.txt,1453,0,0,0,1,1,1,1,1,1,1,7
1899-12-11_t18991211-87-punishment-45.txt,41,0,0,0,0,0,0,0,0,0,0,0
1899-12-11_t18991211-name-7.txt,309,0,0,0,1,1,1,1,1,1,1,7


In [5]:
# tf = df_in[['text_length' , 'drunk*']].sum(axis=1)
tf = df_in.loc[df_in['drunk*']>=1] = df_in['drunk*']/df_in['text_length']
print(tf)

# consider writing a function for each variable here and using .apply to insert the resulting computations in new columns.
# otherwise, load it back into another container (dict of dicts and compute things from there before moving back to dataframe)

#idf = len(df_in['file_id'] / column_values >=1)
#df_in['tf_idf'] = tf*idf

ValueError: Must have equal len keys and value when setting with an iterable